In [1]:
import os, torch, random
from glob import glob
from torch.utils.data import Dataset
import numpy as np
from scipy.signal import fftconvolve
import torch.nn.functional as F

In [2]:
class Dataset(Dataset):
    def __init__(self, train=True, segment=16000):
        super().__init__()
        self.dir = "./train.pt" if train else "./eval.pt"
        self.segment = segment * 5
        self.data = torch.load(self.dir)

    def __len__(self):
        return len(self.data)

    def rir(self, wav: torch.Tensor, rir_path: str) -> torch.Tensor:
        rir_np = np.loadtxt(rir_path)[:, 1]
        rir_energy = np.sum(rir_np**2)
        rir_np = rir_np / np.sqrt(rir_energy)
        rir_tensor = torch.from_numpy(rir_np).float()
        wav_np = wav.cpu().numpy()
        rir_np = rir_tensor.cpu().numpy()
        y_np = fftconvolve(wav_np, rir_np, mode="full")
        y = torch.from_numpy(y_np).float()
        #print(y.shape) # 19199
        return y[0:self.segment]

    def __getitem__(self, idx):
        wav=self.data[idx]
        perr = self.rir(wav, "p-err.txt")

        return wav,perr